In [1]:
from google.cloud import bigquery #need this library to connect the environment to BigQuery
import os #to interact with the underlying operating system of my device 
import altair as alt #data visualization library
import numpy as np #mathematical and logical operations
import pandas as pd #data manipulation and analysis 

## Connect the Jupyterlab environment with BigQuery 

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/felixholmberg/Documents/takehometest-349914-dac2179cc293.json'
bigquery_client = bigquery.Client(project='takehometest-349914')

## Data Visualization 1

Objective of visualization:
- Generate a scatter plot to visualize the relationship between the total number of trips
against the percentage of the population (age 16 and above) without cars by Zip code.

Insights:
- The visualization seems to tell us that there is a relationship between the %age of
people without cars and the total number of bike trips initiated in those zip codes. ○ This makes sense as people who don't have cars are more likely to use
alternative modes of transportation to travel.

## Import the data from BigQuery

In [3]:
QUERY = """
    SELECT start_station_zip_code as zip_code, start_station_landmark as city,
    COUNT(*) AS total_trips, #count the number of trips by Zip Code
    ROUND((MAX(start_station_no_car)/ MAX(start_station_pop_16_over)),4) as no_car_to_population_rate #Get the No Car to Population Rate 
    FROM `takehometest-349914.takehometest.NYC_SF_data_stacked_enhanced_census`
    WHERE start_station_no_car > 0 # to avoid BadRequest: 400 division by zero: 0 / 0 errors
    AND start_station_pop_16_over > 0 # to avoid BadRequest: 400 division by zero: 0 / 0 errors
    GROUP BY start_station_zip_code, start_station_landmark #aggregate by zip code and landmark (city)
 """

dataframe = (bigquery_client.query(QUERY).result().to_dataframe(create_bqstorage_client=True,))
print(dataframe.head())

  zip_code           city  total_trips  no_car_to_population_rate
0    10037  New York City         4783                     0.3617
1    11215  New York City       242758                     0.3407
2    10035  New York City        32832                     0.3169
3    94133  San Francisco        23041                     0.2450
4    10280  New York City       424322                     0.6174


In [4]:
scatter = alt.Chart(dataframe, title ='Relationship between Pct of Population w/o Cars and number of Bike Trips by Zipcode').mark_circle(size=100).encode(
    x=alt.X('no_car_to_population_rate', axis=alt.Axis(labels=True, title="Pct of Population w/o Cars", format="%", ticks=True)),
    y=alt.Y('total_trips', axis=alt.Axis(labels=True, title="Total Trips", ticks=True)),
    size = 'total_trips',
    tooltip=[alt.Tooltip('total_trips', title='# Trips'),
            alt.Tooltip('no_car_to_population_rate', title='Pct of Population w/o Cars', format=".2%"), 
            alt.Tooltip('zip_code', title ='Zip Code')],
    color=alt.Color('no_car_to_population_rate', title="Color Coding based on Pct",
                   scale=alt.Scale(range=['red', 'darkgreen']))).interactive().properties(width=600, height=400)

scatter.configure_title(fontSize=14)
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

## Data visualization 2

Objective of visualization:
- The purpose of this visualization is to get a better understanding of when the most
amount of bike trips occur throughout the year (January to December) and throughout each Day (00:00 to 23:59)

Insights trips by Month of the Year:
- If we view the graph from a monthly perspective, we will get a solid understanding of
when most bike trips occur throughout the year.
- The graph is clearly displaying a cluster with a lot of bike activity from April to
September. This trend makes sense as these months are generally warmer.

Insights trips by Time Of The Day:
- The visualization shows that there are two major clusters for when the most bike trip
activity occurs throughout the day.
- Looking at when most bike trips are initiated, it clearly tells us that the most bike
rides are initiated from around 16:00 to 19:00 in the afternoon to evening, or from 07:00 to 09:00 in the morning.
- This trend makes sense as these are normally the hours of the day when a majority of people have to go to work, or get out of work.


## Import the data from BigQuery

In [5]:
QUERY2 = """
    SELECT CONCAT(FORMAT_DATE("%m", start_date), ", ", FORMAT_DATE("%B", start_date)) AS month,  
       EXTRACT(HOUR FROM start_date) AS hour,
       COUNT(*) AS total_trips
    FROM `takehometest-349914.takehometest.NYC_SF_data_stacked_enhanced_census`
    GROUP BY CONCAT(FORMAT_DATE("%m", start_date), ", ", FORMAT_DATE("%B", start_date)),
        EXTRACT(HOUR FROM start_date)
 """

dataframe2 = (bigquery_client.query(QUERY2).result().to_dataframe(create_bqstorage_client=True,))
print(dataframe2.head())

           month  hour  total_trips
0      04, April    18       466363
1  09, September    18       640900
2      03, March     0       101791
3       07, July     8       420620
4      04, April    10       209493


In [6]:
heatmap = alt.Chart(dataframe2, title = 'Relationship between Month and number of bike trips by hour of the day').mark_rect(size=100).encode(
    x = alt.X('hour:O', #ordinal
             axis=alt.Axis(labels=True, title="Hour of the Day", ticks=True)),
    y = alt.Y('month:O', #ordinal
              axis=alt.Axis(labels=True, title="", ticks=True),
              sort=alt.SortField(
              field='month',
              order="descending")),
    color = alt.Color('total_trips', title='Total Trips'),
    tooltip=[alt.Tooltip('total_trips', title='# Trips'),
            alt.Tooltip('month', title='Month'),
            alt.Tooltip('hour', title='Hour')]).interactive().properties(width=600, height=450)
heatmap.configure_title(fontSize=14)
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

## Data visualization 3

Objective of visualization:
- The purpose of this visualization is to compare the frequency of bike rides throughout the
week, and to see how bike trip behavior might differ between New York City and San Francisco.

Insights:
- The chart tells us that both cities are experiencing a drop-off in activity during the
weekend. Furthermore, one observation that is interesting for both cities, is that the average duration of trips see an increase over the weekend. This is likely due to the fact that people on the weekends may be more likely to use bikes for leisure, while users during the week might use the bike to get around the city quickly for work or get from point A to B.





In [7]:
QUERY3 = """
    SELECT CONCAT(FORMAT_DATE('%u', start_date), ', ', FORMAT_DATE('%A', start_date))  AS weekday_name_full, 
        start_station_landmark AS city, 
        ROUND((SUM(trip_duration)-MAX(trip_duration))/(COUNT(trip_duration)-1)/60,1) AS avg_trip_duration_mins,
        COUNT(*) AS total_trips
    FROM `takehometest-349914.takehometest.NYC_SF_data_stacked_enhanced_census`
    WHERE start_station_landmark in("San Francisco", "New York City")
    GROUP BY CONCAT(FORMAT_DATE('%u', start_date), ', ', FORMAT_DATE('%A', start_date)), start_station_landmark
 """

dataframe3 = (bigquery_client.query(QUERY3).result().to_dataframe(create_bqstorage_client=True,))
print(dataframe3.head())

  weekday_name_full           city  avg_trip_duration_mins  total_trips
0         1, Monday  New York City                    14.9      7407881
1         5, Friday  San Francisco                    15.4       139476
2       6, Saturday  San Francisco                    35.7        53315
3       4, Thursday  New York City                    14.7      8007790
4         5, Friday  New York City                    15.3      7623992


In [8]:
nyc = dataframe3[dataframe3['city']=='New York City'] #create one df for NYC
sf = dataframe3[dataframe3['city']=='San Francisco'] #create one df for SF

base_sf = alt.Chart(sf, title='SF Bike Trips Insights').encode(
    alt.X('weekday_name_full', axis=alt.Axis(title=None))
)

bar_sf = base_sf.mark_bar(opacity=0.4, color='#57A44C', size=20).transform_fold(
    fold=['total_trips'], 
    as_=['variable', 'value']
).encode(
    alt.Y('total_trips', axis=alt.Axis(title='Total Number of Trips')), tooltip=[alt.Tooltip('total_trips', title='Total Number of Trips')]
)

line_sf = base_sf.mark_line(stroke='#5276A7', interpolate='monotone', point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y('avg_trip_duration_mins',axis=alt.Axis(title='Avg. Duration (mins)')),   tooltip=[alt.Tooltip('avg_trip_duration_mins', title='Avg. Duration (mins)')]
)

chart_sf = alt.layer(bar_sf, line_sf).resolve_scale(y='independent').properties(width=350, height=200)

base_nyc = alt.Chart(nyc, title='NYC Bike Trips Insights').encode(
    alt.X('weekday_name_full', axis=alt.Axis(title=None))
)

bar_nyc = base_nyc.mark_bar(opacity=0.4, color='#57A44C', size=20).encode(
    alt.Y('total_trips', axis=alt.Axis(title='Total Number of Trips')), tooltip=[alt.Tooltip('total_trips', title='Total Number of Trips')]
)

line_nyc = base_nyc.mark_line(stroke='#5276A7', interpolate='monotone', point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y('avg_trip_duration_mins',axis=alt.Axis(title='Avg. Duration (mins)')),   tooltip=[alt.Tooltip('avg_trip_duration_mins', title='Avg. Duration (mins)')]
)

chart_nyc = alt.layer(bar_nyc, line_nyc).resolve_scale(y='independent').properties(width=350, height=200)

alt.vconcat(chart_nyc, chart_sf)
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')